In [24]:
# 加载模型
import torch
from stable_baselines3 import DQN
import pandas as pd

# model = DQN.load("./model/SACE_dqn_model",device='cpu')
# model_lane_change = model.policy.q_net.q_net
model_path = "./model/model_test1"
device = 'cpu'  # 如果需要在 GPU 上推断，可以改为 'cuda'
model = DQN.load(model_path, device=device)
model_lane_change = model.policy.q_net.q_net
# data = torch.load("./data/SACE_dqn_data.pth")
# data_x = torch.stack(data)

In [25]:
data_pd = pd.read_csv("./data/obs_data.csv")
data_tensor = torch.tensor(data_pd.values, dtype=torch.float32)

# 从 tensor_data 中随机选择 1000 个样本
num_samples_to_select = 1000
data_x = data_tensor[0:999]

In [26]:
# import pandas as pd
# import shap
# import matplotlib.font_manager as fm
# import matplotlib
# matplotlib.use('TkAgg')  # 或其他支持动画的后端，如 'Qt5Agg'
# import pickle
# import pandas as pd
# import shap
# # 设置 Matplotlib 的默认样式
# import matplotlib.pyplot as plt
# plt.style.use('default')
# plt.rcParams['savefig.format'] = 'svg'
#
# # 设置 Matplotlib 的默认背景色为白色
# plt.rcParams['axes.facecolor'] = 'white'

In [27]:
from FullyNet_Shapleyflowtest import FullyNet_Shapleyflow

In [28]:
model_lane_change(data_x)

tensor([[4.1519, 4.4522, 4.4544],
        [4.0970, 4.3501, 4.2724],
        [3.9330, 1.1381, 1.1019],
        ...,
        [3.6503, 3.7753, 3.9040],
        [3.7951, 3.9057, 3.9922],
        [3.7543, 3.9404, 3.8586]], grad_fn=<AddmmBackward0>)

In [29]:
# max_indices = torch.argmax(model_lane_change(data_x), dim=1)
# 找到每组输出的最大值及其索引
max_values, max_indices = torch.max(model_lane_change(data_x), dim=1) # 所有被选择的决策索引，及其输出值
# 找到最大值中的最小值的索引
min_indices_in_max_values = torch.argmin(max_values)  # 输出值最小的索引，正定基值点索引

In [30]:
background = data_x[min_indices_in_max_values:min_indices_in_max_values+1]

In [31]:
test_aa = FullyNet_Shapleyflow(model_lane_change, background)
aaaaa = test_aa.calculate_shapley_flow(data_x)

## 绘图

### 局部解释图

#### 局部减速解释

In [32]:
# 找到元素是 0 的索引
indices_0 = torch.nonzero(max_indices == 0).squeeze()

# 找到元素是 1 的索引
indices_1 = torch.nonzero(max_indices == 1).squeeze()

# 找到元素是 2 的索引
indices_2 = torch.nonzero(max_indices == 2).squeeze()

In [33]:
import shap
sample_index = indices_0[0]   # 样本索引
out_index = 0      # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_lane_change(background)[0][out_index].detach().numpy()

In [34]:
expl = shap.Explanation(values=shapley_value, base_values=base_value.item())
expl.data = data_x[sample_index].numpy()
obs_name = ['$\Delta d$', '$\Delta v$']
expl.feature_names = obs_name

In [35]:
import matplotlib
matplotlib.use('TkAgg')  # 或其他支持动画的后端，如 'Qt5Agg'
import pickle
import pandas as pd
import shap
# 设置 Matplotlib 的默认样式
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['savefig.format'] = 'svg'
%matplotlib inline
shap.waterfall_plot(expl, show=False)
plt.savefig('figure/局部减速.svg',dpi=600, bbox_inches='tight')
plt.close()
# plt.cla()

#### 局部保持解释

In [36]:
sample_index = indices_1[0]  # 样本索引
out_index = 1  # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_lane_change(background)[0][out_index].detach().numpy()
expl = shap.Explanation(values=shapley_value, base_values=base_value.item())
expl.data = data_x[sample_index].numpy()
expl.feature_names = obs_name


In [37]:
plt.style.use('default')
plt.rcParams['savefig.format'] = 'svg'
%matplotlib inline
shap.waterfall_plot(expl, show=False)
plt.savefig('figure/局部保持.svg', dpi=600, bbox_inches='tight')
plt.close()
# plt.cla()

#### 局部加速解释

In [38]:
sample_index = indices_2[0]  # 样本索引
out_index = 2  # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_lane_change(background)[0][out_index].detach().numpy()
expl = shap.Explanation(values=shapley_value, base_values=base_value.item())
expl.data = data_x[sample_index].numpy()
expl.feature_names = obs_name


plt.style.use('default')
plt.rcParams['savefig.format'] = 'svg'
%matplotlib inline
shap.waterfall_plot(expl, show=False)
plt.savefig('figure/局部加速.svg', dpi=600, bbox_inches='tight')
plt.close()

In [39]:
sample_index = indices_1[0]  # 样本索引
out_index = 1  # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_lane_change(background)[0][out_index].detach().numpy()

base_value

array(1.8108337, dtype=float32)

In [40]:
base_value.item()

1.8108336925506592

In [45]:
model_lane_change(background)

tensor([[3.5513, 1.8108, 0.1044]], grad_fn=<AddmmBackward0>)

### 全局解释图

In [46]:
stacked_tensor = torch.stack(aaaaa)
obs_name = ['$d$', '$\Delta v$']
shap.summary_plot(stacked_tensor[indices_0, 0,:].numpy(), data_x[indices_0].numpy(), feature_names=obs_name, show=False)
plt.savefig('figure/减速情况1.svg',dpi=600, bbox_inches='tight')
plt.close()

In [47]:

shap.summary_plot(stacked_tensor[indices_1, 1, :].numpy(), data_x[indices_1].numpy(), feature_names=obs_name, show=False)
plt.savefig('figure/保持情况1.svg',dpi=600, bbox_inches='tight')
plt.close()

In [48]:
stacked_tensor = torch.stack(aaaaa)

shap.summary_plot(stacked_tensor[indices_2, 2, :].numpy(), data_x[indices_2].numpy(), feature_names=obs_name, show=False)
plt.savefig('figure/加速情况1.svg',dpi=600, bbox_inches='tight')
plt.close()